In [122]:
# import libraries
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

In [123]:
# specifications
dbname = "fdac18mp2" #please use this database
collname = "glprj_eherron5"
my_char = 'o'

# beginning page index
begin = "0"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]

In [124]:
# urls
gl_url = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"
sf_url = "https://sourceforge.net/directory/?q="

In [125]:
gleft = 0

header = {'per_page': 99}

In [126]:
def url_exists(url):
    request = requests.get(url)
    return request.status_code == 200

In [127]:
# check remaining query chances for rate-limit restriction
def gl_wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

In [128]:
# send queries and extract urls - gitlab
def gl_get(url, coll, start_char):

    global gleft
    global header
    global bginnum
    gleft = gl_wait(gleft)
    values = []
    size = 0

    max_links = 50
    links_count = 0
    
    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)

            for el in array:
                proj_name = el['path'].lower()
                proj_url = el['http_url_to_repo']
                if proj_name.startswith(start_char) and url_exists(proj_url):
                    print('inserting url', links_count, ' for path', el['path'], proj_url)
                    coll.insert(el)
                    links_count += 1
                    if links_count >= max_links:
                        return

            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = gl_wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()

                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            coll.insert(el)
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')

In [ ]:
#start retrieving gitlab
gl_get(gl_url,coll, my_char)

inserting url 0  for path openmw https://gitlab.com/Fynjyfun/openmw.git


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [ ]:
def sf_get(url, start_char):
    links_count = 0
    max_links = 50
    page_num = 1
    rest = "http://sourceforge.net/rest/p/"
    
    while url_exists(url+ str(page_num)):
        r = requests.get(url+ str(page_num))
        
        # gets html text
        text = r.text
        soup = BeautifulSoup(text, 'html.parser')
        
        # find all projects listed on page
        for item in soup.find_all(class_="result-heading-texts"):
            
            a = item.find('a')
            link = a['href']
            name = link.split('/')[1]
            title = a.get_text()

            if title.lower().startswith(start_char) and url_exists(rest + name):
                coll.insert_one(requests.get(rest + name).json())
                links_count += 1
                if links_count >= max_links:
                    return
        page_num += 1
    return


In [ ]:
# get 50 source forge projects
sf_get(sf_url, my_char)